In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

##### Merge: periodos + clientes + productos

In [3]:

sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")
del sellin, productos, stocks

df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

periodos = pd.date_range(start=df['periodo_dt'].min(), end=df['periodo_dt'].max(), freq="MS")
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()

idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
completo["periodo"] = completo["periodo"].dt.strftime("%Y%m").astype(int)

del periodos, productos, clientes
gc.collect()
completo.head()

Ventas-Productos-Stocks: 2945818 filas y 13 columnas


,product_id,customer_id,periodo
0,20524,10234,201701
1,20524,10234,201702
2,20524,10234,201703
3,20524,10234,201704
4,20524,10234,201705


##### Cruzamos con productos

In [5]:
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
completo = completo.merge(productos, how='left', on="product_id")
del productos
gc.collect()

18

##### Cruzamos con stock

In [6]:
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
completo = completo.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks
gc.collect()
completo.head()

,product_id,customer_id,periodo,cat1_x,cat2_x,cat3_x,brand_x,sku_size_x,cat1_y,cat2_y,cat3_y,brand_y,sku_size_y,stock_final
0,20524,10234,201701,HC,VAJILLA,Cristalino,Importado,500.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,20524,10234,201702,HC,VAJILLA,Cristalino,Importado,500.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,20524,10234,201703,HC,VAJILLA,Cristalino,Importado,500.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,20524,10234,201704,HC,VAJILLA,Cristalino,Importado,500.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,20524,10234,201705,HC,VAJILLA,Cristalino,Importado,500.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN


##### Cruzamos con ventas

In [7]:
sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
# Agrupar ventas por periodo, cliente y producto
dt = sellin.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
df_completo = completo.merge(dt, how='left', on=['periodo', 'customer_id','product_id'])
df_completo['tn'] = df_completo['tn'].fillna(0)
del sellin, dt, completo
gc.collect()

40

##### Target

In [ ]:
# Asegurarte de tener 'periodo_dt' (datetime) en completo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear DataFrame auxiliar con tn como target y fecha adelantada
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras['periodo_target_dt'] = ventas_futuras['periodo_dt'] - pd.DateOffset(months=2)
ventas_futuras = ventas_futuras.rename(columns={'tn': 'target'})

# Merge con completo usando periodo adelantado
df_completo = df_completo.merge(
    ventas_futuras[['periodo_target_dt', 'customer_id', 'product_id', 'target']],
    how='left',
    left_on=['periodo_dt', 'customer_id', 'product_id'],
    right_on=['periodo_target_dt', 'customer_id', 'product_id']
)

# Eliminar columna auxiliar
df_completo = df_completo.drop(columns=['periodo_target_dt'])
del ventas_futuras
gc.collect()
print(f"✅ Target generado. Filas con target no nulo: {df_completo['target'].notna().sum()}")

✅ Target generado. Filas con target no nulo: 25027434


##### Verifico las NaN en el target: Existen porque hay clientes que solo compraron 2 veces.

In [10]:
nan_count = df_completo['target'].isna().sum()
print(f"🔍 Total de NaN en target: {nan_count}")
del nan_count
gc.collect()

🔍 Total de NaN en target: 1472202


0

##### Generación de IDs

In [11]:
df_completo = df_completo.sort_values(['periodo', 'customer_id', 'product_id'])
df_completo['id'] = df_completo.groupby(['customer_id', 'product_id']).cumcount() + 1

##### Periodo 

In [19]:
df_completo["periodo_dt"] = pd.to_datetime(df_completo["periodo"].astype(str), format="%Y%m")

##### Eliminar productos que no nacieron

In [12]:
nacimiento_producto = df.groupby("product_id")["periodo_dt"].agg(["min"]).reset_index()
# Renombrar columna max a muerte_cliente_dt
nacimiento_producto = nacimiento_producto.rename(columns={'min': 'nacimiento_producto'})


# Unir con df_final para traer fecha de muerte del cliente
df_completo = df_completo.merge(nacimiento_producto, on='product_id', how='left')

# Filtrar filas donde periodo_dt > muerte_cliente_dt
df_completo = df_completo[df_completo['periodo_dt'] >= df_completo['nacimiento_producto']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['muerte_cliente_dt'])
del nacimiento_producto
gc.collect()
print(f"✅ Dataset filtrado con {len(df_completo):,} filas.")

✅ Dataset filtrado con 21,425,136 filas.


##### Eliminamos clientes que no nacieron

In [13]:
# df_final["periodo_dt"] = pd.to_datetime(df_final["periodo"].astype(str), format="%Y%m")

nacimiento_cliente = df.groupby("customer_id")["periodo_dt"].agg(["min"]).reset_index()

# Renombrar columna max a nacimiento_cliente_dt
nacimiento_cliente = nacimiento_cliente.rename(columns={'min': 'nacimiento_cliente_dt'})

# Unir con df_final para traer fecha de muerte del cliente
df_completo = df_completo.merge(nacimiento_cliente, on='customer_id', how='left')

# Filtrar filas donde periodo_dt > nacimiento_cliente_dt
df_completo = df_completo[df_completo['periodo_dt'] >= df_completo['nacimiento_cliente_dt']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['nacimiento_cliente_dt'])
del nacimiento_cliente
gc.collect()

print(f"✅ Dataset filtrado con {len(df_completo):,} filas.")


✅ Dataset filtrado con 19,639,107 filas.


##### Correlograma

In [15]:
# Supongamos que tu DataFrame se llama df
cor_matrix = df_completo.corr(numeric_only=True)

# Tomamos solo la parte superior de la matriz (sin la diagonal)
upper = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))

# Filtramos correlaciones fuertes (valor absoluto mayor a 0.7)
high_corr = upper.stack().reset_index()
high_corr.columns = ['Variable 1', 'Variable 2', 'Correlación']
high_corr_filtrada = high_corr[high_corr['Correlación'].abs() > 0.7]

print(high_corr_filtrada)

del high_corr_filtrada, cor_matrix, upper, high_corr
gc.collect()

    Variable 1       Variable 2  Correlación
29     periodo               id     0.955593
30  sku_size_x       sku_size_y     1.000000
51          tn  cust_request_tn     0.993594


47

##### Elimino variables muy correlacionadas

In [16]:
df_completo.drop(columns=['cust_request_tn', 'periodo', 'sku_size_y'], inplace=True)

##### Extracción de componentes temporales

In [17]:
df_completo['year'] = df_completo['periodo_dt'].dt.year
df_completo['month'] = df_completo['periodo_dt'].dt.month
# Variables dummy estacionales
df_completo['quarter'] = df_completo['periodo_dt'].dt.quarter
df_completo['semester'] = np.where(df_completo['month'] <= 6, 1, 2)
# Efectos de fin de año
df_completo['year_end'] = np.where(df_completo['month'].isin([11, 12]), 1, 0)
df_completo['year_start'] = np.where(df_completo['month'].isin([1, 2]), 1, 0)
# Indicadores estacionales
df_completo['season'] = df_completo['month'] % 12 // 3 + 1  # 1:Invierno, 2:Primavera, etc.
# Variables cíclicas (para capturar patrones estacionales)
df_completo['month_sin'] = np.sin(2 * np.pi * df_completo['month']/12)
df_completo['month_cos'] = np.cos(2 * np.pi * df_completo['month']/12)

#####  Lags, diferencias, medias móviles y otras yerbas

In [ ]:
# # Aseguramos el orden por grupo y tiempo
# df_completo = df_completo.sort_values(['customer_id', 'product_id', 'periodo_dt'])

# # Definimos la función que genera todas las variables para cada grupo
# def generar_features(grupo):
#     # Lags
#     for i in range(1, 13):
#         grupo[f'lag_{i}'] = grupo['target'].shift(i)
#         grupo[f'delta_{i}'] = grupo['target'].diff(i)
#         grupo[f'pct_change_{i}'] = grupo['target'].pct_change(i)

#     # Rolling statistics
#     windows = [2, 3, 6, 9, 12]
#     for w in windows:
#         grupo[f'rolling_mean_{w}'] = grupo['target'].rolling(window=w, min_periods=1).mean()
#         grupo[f'rolling_std_{w}'] = grupo['target'].rolling(window=w, min_periods=1).std()
#         grupo[f'rolling_min_{w}'] = grupo['target'].rolling(window=w, min_periods=1).min()
#         grupo[f'rolling_max_{w}'] = grupo['target'].rolling(window=w, min_periods=1).max()
#         grupo[f'rolling_median_{w}'] = grupo['target'].rolling(window=w, min_periods=1).median()

#     # Tendencia acumulada
#     grupo['expanding_mean'] = grupo['target'].expanding().mean()
#     grupo['cumulative_sum'] = grupo['target'].cumsum()

#     # Diferencia estacional y comparación interanual
#     grupo['seasonal_diff_12'] = grupo['target'].diff(12)
#     grupo['vs_prev_year'] = grupo['target'] / grupo['target'].shift(12) - 1

#     # Descomposición simple
#     grupo['trend'] = grupo['target'].rolling(window=12, min_periods=1).mean()
#     grupo['seasonality'] = grupo['target'] - grupo['trend']

#     # Eventos especiales
#     grupo['new_high'] = (grupo['target'] == grupo[f'rolling_max_12']).astype(int)
#     grupo['new_low'] = (grupo['target'] == grupo[f'rolling_min_12']).astype(int)

#     # Aceleración
#     grupo['acceleration'] = grupo['target'].diff(1).diff(1)  # o bien: delta_1.diff(1)
    
#     return grupo

# # Aplicamos la función por grupo
# df_completo = df_completo.groupby(['customer_id', 'product_id'], group_keys=False).apply(generar_features)


In [ ]:
df_completo = df_completo.sort_values(['periodo_dt', 'customer_id', 'product_id'])

In [18]:
for i in range(1, 13):
    df_completo[f'lag_{i}'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.shift(i))

KeyboardInterrupt: 

In [ ]:
for i in range(1, 13):
    df_completo[f'delta_{i}'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.diff(i))

In [ ]:
for i in range(1, 13):
    df_completo[f'pct_change_{i}'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.pct_change(i))

In [ ]:
# Rolling statistics
windows = [2, 3, 6, 9, 12]
for w in windows:
    df_completo[f'rolling_mean_{w}'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.rolling(window=w, min_periods=1).mean())

In [ ]:
# Rolling statistics
windows = [2, 3, 6, 9, 12]
for w in windows:
    df_completo[f'rolling_std_{w}'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.rolling(window=w, min_periods=1).std())

In [ ]:
# Rolling statistics
windows = [2, 3, 6, 9, 12]
for w in windows:
    df_completo[f'rolling_median_{w}'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.rolling(window=w, min_periods=1).median())

In [ ]:
# Tendencia acumulada
df_completo[f'expanding_mean'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.expanding().mean())
df_completo[f'cumulative_sum'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.cumsum())

In [ ]:
# Diferencia estacional y comparación interanual
df_completo[f'seasonal_diff_12'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.diff(12))
df_completo[f'vs_prev_year'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x / x.shift(12) - 1)

In [ ]:
# Descomposición simple
df_completo[f'trend'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())
df_completo['seasonality'] = df_completo['target'] - df_completo['trend']

In [ ]:
# Eventos especiales   
df_completo['new_high'] = (df_completo['target'] == df_completo['rolling_max_12']).astype(int)
df_completo['new_low'] = (df_completo['target'] == df_completo['rolling_min_12']).astype(int)

In [ ]:
# Acelaración
df_completo[f'trend'] = df_completo.groupby(['customer_id', 'product_id'])['target'].transform(lambda x: x.diff(1).diff(1)) # o bien: delta_1.diff(1)